In [1]:
from DDCASPT2 import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm
# Geom manipulate
from AaronTools.geometry import Geometry
from tqdm import tqdm
from time import perf_counter
import seaborn as sns

In [2]:
done = set([i for i in sorted(os.listdir()) if os.path.exists(os.path.join(i,i+".csv"))])

In [3]:
dirs = []
for i in sorted(os.listdir()):
    try:
        r = float(i)
        dirs.append(i)
    except:
        print("Nope",i)

Nope .ipynb_checkpoints
Nope DDCASPT2.py
Nope FeO.xyz
Nope __pycache__
Nope energies.csv
Nope newddcaspt2.ipynb
Nope plot.py
Nope plot_view.sh
Nope resubmitall.sh
Nope runall.sh
Nope submitall.sh
Nope test_ind.pickle
Nope train_ind.pickle


In [4]:
# radius_range_dirs = []
# for i in glob('*.*'):
#     try:
#         radstr = float(i)
#         if radstr>=1.4:
#             radius_range_dirs.append(i)
#     except:
#         continue

# radius_range = sorted(radius_range_dirs)

# train_ind,test_ind=radius_range[0::2],radius_range[1::2]

# print(len(train_ind),len(test_ind))
# with open('train_ind.pickle', 'wb') as handle:
#     pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'wb') as handle:
#     pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'rb') as handle:
#     test_ind = pickle.load(handle)

# with open('train_ind.pickle', 'rb') as handle:
#     train_ind = pickle.load(handle)
    
# print(len(train_ind),len(test_ind))    

In [5]:
radius_range=np.linspace(1.4,8,661)

radius_range=radius_range[radius_range<3][0::2]

# train_ind,test_ind=radius_range[0::2],radius_range[1::2]
# train_test_split(radius_range, test_size=0.3, random_state=0)
# print(len(train_ind),len(test_ind))
# with open('train_ind.pickle', 'wb') as handle:
#     pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'wb') as handle:
#     pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'rb') as handle:
#     test_ind = pickle.load(handle)

# with open('train_ind.pickle', 'rb') as handle:
#     train_ind = pickle.load(handle)
    
# print(len(train_ind),len(test_ind))    

In [6]:
len(radius_range)

80

In [7]:
def gen_run(r,path):
    with open(os.path.join(path,f'run.sh'),'w') as g:
        g.write(f"""#!/bin/bash
#This file is a submission script to request the ISAAC resources from Slurm
#SBATCH --account=ACF-UTK0011             # The project account to be charged
#SBATCH --job-name=ironoxo_{r}		       #The name of the job
#SBATCH --nodes=1                     # Number of nodes
#SBATCH --ntasks-per-node=4          # cpus per node
#SBATCH --partition=campus            # If not specified then default is "campus"
#SBATCH --qos=campus
#SBATCH --time=0-04:00:00             # Wall time (days-hh:mm:ss)
#SBATCH --error=job.e%J	       # The file where run time errors will be dumped
#SBATCH --output=job.o%J	       # The file where the output of the terminal will be dumped



# PATCH!
# MOVE TO WORKING DIR
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
pwd
# DEFINE WorkDir
mkdir $SLURM_SUBMIT_DIR/$SLURM_JOBID
export WorkDir=$SLURM_SUBMIT_DIR/$SLURM_JOBID
export MOLCAS_WORKDIR="/lustre/isaac/scratch/gjones39"
export MOLCAS="/lustre/isaac/proj/UTK0022/Grier2025/Test/build"
echo $WorkDir



module purge
module load intel-compilers/2021.2.0
module load hdf5/1.10.8-intel
module load cmake/3.23.2-intel

# DEFINE WorkDir
echo "Starting at $(date)"
echo "Running on hosts: $SLURM_NODELIST"
echo "Running on $SLURM_NNODES nodes."
echo "Running $SLURM_NTASKS tasks."
echo "Current working directory is $(pwd)"

# THE COMMAND
pymolcas --new --clean {r}.input -oe {r}.output
# CLEAN-UP AND EXIT
rm -r $SLURM_SUBMIT_DIR/$SLURM_JOBID
echo "Program finished with exit code $? at: $(date)"
""")    

In [8]:

topdir = os.getcwd()



In [9]:
t0 = perf_counter()
geom = Geometry('FeO.xyz')
oxo, iron = geom.atoms[0],geom.atoms[1]

for idxr, r in tqdm(enumerate(radius_range)):

# Create radius subdirectory
    radstr = f"{r:.2f}"
    rad_dir = os.path.join(os.getcwd(),radstr)
    if os.path.exists(rad_dir)==False:
        os.mkdir(rad_dir)
    
        
    # Write xyz
    geom.change_distance(iron,oxo,r)
    geom.write(os.path.join(rad_dir,radstr))
    # if idxr==0:
    #     scfprevious=None
    #     d = DDCASPT2(rad_dir,'ANO-RCC-VTZP',radstr,10,14,None,scf_previous=scfprevious,casscf_previous=scfprevious,symmetry=1,spin=4,UHF=True,CIROOT="3 3 1",frozen=0,pt2maxiter=50,MSroots=3,charge=2)(inputwrite=True,run=True,feat=False)
    # else:  
    if os.path.exists(os.path.join(topdir,radstr,"GMJ_RHS_H_P_3_.csv"))==False:
        print(os.path.join(topdir,radstr,"GMJ_RHS_H_P_3_.csv"),"Doesn't exist... running")
        scfprevious=os.path.join(topdir,f"{radius_range[idxr-1]:.2f}",f"{radius_range[idxr-1]:.2f}.UhfOrb")        
        casscf_previous=os.path.join(topdir,f"{radius_range[idxr-1]:.2f}",f"{radius_range[idxr-1]:.2f}.RasOrb")
        d = DDCASPT2(rad_dir,'ANO-RCC-VTZP',radstr,10,14,None,scf_previous=scfprevious,casscf_previous=casscf_previous,symmetry=1,spin=4,UHF=True,CIROOT="3 3 1",frozen=0,pt2maxiter=50,MSroots=3,charge=2)(inputwrite=True,run=True,feat=False)
    
    gen_run(radstr,rad_dir)
print(perf_counter()-t0)

0it [00:00, ?it/s]

/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/MScaspt2iron/1.84/GMJ_RHS_H_P_3_.csv Doesn't exist... running
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/


23it [02:41,  7.01s/it]

/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/MScaspt2iron/1.86/GMJ_RHS_H_P_3_.csv Doesn't exist... running
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/


24it [04:39, 13.49s/it]

...

/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/MScaspt2iron/2.96/GMJ_RHS_H_P_3_.csv Doesn't exist... running
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/


79it [09:25,  5.04s/it]

/home/grierjones/DDCASPT2/iron_oxo_DDCASPT2/MScaspt2iron/2.98/GMJ_RHS_H_P_3_.csv Doesn't exist... running
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/


80it [09:30,  7.14s/it]

570.8648840860697


In [10]:
sns.color_palette('Paired').as_hex()[1]

'#1f78b4'